## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append('..')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/weatherpy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mahebourg,MU,2022-07-07 16:10:49,-20.4081,57.7000,70.09,68,75,16.11,broken clouds
1,1,Verkhniy Baskunchak,RU,2022-07-07 16:10:49,48.2256,46.7217,85.17,23,0,6.53,clear sky
2,2,Westport,US,2022-07-07 16:10:41,41.1415,-73.3579,82.72,59,56,4.00,broken clouds
3,3,Kem,RU,2022-07-07 16:10:49,64.9555,34.5793,59.94,72,100,7.31,overcast clouds
4,4,Namibe,AO,2022-07-07 16:10:50,-15.1961,12.1522,67.87,80,4,8.75,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Verkhniy Baskunchak,RU,2022-07-07 16:10:49,48.2256,46.7217,85.17,23,0,6.53,clear sky
2,2,Westport,US,2022-07-07 16:10:41,41.1415,-73.3579,82.72,59,56,4.00,broken clouds
9,9,Jamestown,US,2022-07-07 16:10:51,42.0970,-79.2353,80.02,64,75,8.05,broken clouds
12,12,Surt,LY,2022-07-07 16:10:51,31.2089,16.5887,84.85,61,0,9.69,clear sky
17,17,Wisconsin Rapids,US,2022-07-07 16:10:53,44.3836,-89.8174,77.29,71,20,5.75,few clouds
20,20,La Vega,DO,2022-07-07 16:10:54,19.2221,-70.5296,88.65,42,5,13.06,clear sky
21,21,Sao Miguel Do Iguacu,BR,2022-07-07 16:10:54,-25.3481,-54.2378,81.48,40,60,8.97,broken clouds
29,29,Axim,GH,2022-07-07 16:10:56,4.8699,-2.2405,78.64,78,86,10.36,overcast clouds
30,30,Beberibe,BR,2022-07-07 16:10:56,-4.1797,-38.1306,82.63,66,99,12.80,overcast clouds
34,34,Arraial Do Cabo,BR,2022-07-07 16:10:58,-22.9661,-42.0278,82.26,51,0,12.66,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Verkhniy Baskunchak,RU,2022-07-07 16:10:49,48.2256,46.7217,85.17,23,0,6.53,clear sky
2,2,Westport,US,2022-07-07 16:10:41,41.1415,-73.3579,82.72,59,56,4.00,broken clouds
9,9,Jamestown,US,2022-07-07 16:10:51,42.0970,-79.2353,80.02,64,75,8.05,broken clouds
12,12,Surt,LY,2022-07-07 16:10:51,31.2089,16.5887,84.85,61,0,9.69,clear sky
17,17,Wisconsin Rapids,US,2022-07-07 16:10:53,44.3836,-89.8174,77.29,71,20,5.75,few clouds
...,...,...,...,...,...,...,...,...,...,...,...
670,670,Waling,NP,2022-07-07 16:14:20,27.9833,83.7667,78.39,83,64,1.21,broken clouds
671,671,Koygorodok,RU,2022-07-07 16:14:20,60.4450,50.9968,76.44,67,86,3.78,overcast clouds
673,673,Limbe,CM,2022-07-07 16:14:21,4.0242,9.2149,78.66,84,67,8.63,light rain
675,675,Torres,BR,2022-07-07 16:14:21,-29.3353,-49.7269,76.82,71,100,3.18,overcast clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Verkhniy Baskunchak,RU,85.17,clear sky,48.2256,46.7217,
2,Westport,US,82.72,broken clouds,41.1415,-73.3579,
9,Jamestown,US,80.02,broken clouds,42.0970,-79.2353,
12,Surt,LY,84.85,clear sky,31.2089,16.5887,
17,Wisconsin Rapids,US,77.29,few clouds,44.3836,-89.8174,
20,La Vega,DO,88.65,clear sky,19.2221,-70.5296,
21,Sao Miguel Do Iguacu,BR,81.48,broken clouds,-25.3481,-54.2378,
29,Axim,GH,78.64,overcast clouds,4.8699,-2.2405,
30,Beberibe,BR,82.63,overcast clouds,-4.1797,-38.1306,
34,Arraial Do Cabo,BR,82.26,clear sky,-22.9661,-42.0278,


In [12]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the latitiude and the longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the result and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Verkhniy Baskunchak,RU,85.17,clear sky,48.2256,46.7217,Kombat
2,Westport,US,82.72,broken clouds,41.1415,-73.3579,The Westport Inn
9,Jamestown,US,80.02,broken clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
12,Surt,LY,84.85,clear sky,31.2089,16.5887,الهادي ابوشعاله سرت
17,Wisconsin Rapids,US,77.29,few clouds,44.3836,-89.8174,Hotel Mead & Conference Center


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))